<a href="https://colab.research.google.com/github/Ritishh00/GEN-AI/blob/main/GEN_AI_29_01_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers, Model
from tensorflow.keras import datasets
from tensorflow.keras.losses import MeanSquaredError , KLDivergence , BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
# Import ModelCheckpoint and EarlyStopping from tensorflow.keras.callbacks instead of tensorflow.keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from numpy import cov, trace, iscomplexobj, asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from skimage.transform import resize

In [9]:
from tensorflow.keras.datasets import cifar10  # Import the cifar10 dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()  # Load the CIFAR-10 dataset

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [11]:
x_train = x_train.astype('float32') / 255.0  # Normalize the training data
x_test = x_test.astype('float32') / 255.0  # Normalize the testing data

In [13]:
latent_dim = 1024
image_shape = x_train[0].shape  # Get the shape directly using .shape
mse_loss = MeanSquaredError()
kl_loss = KLDivergence()

In [25]:
class VAE(Model):
    def __init__(self, latent_dim, image_shape, beta):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.image_shape = image_shape
        self.beta = beta

        #encoder
        self.encoder = keras.Sequential([
            layers.Input(shape=image_shape, name="Encoder_input_layer"),
            layers.Conv2D(32, 3, strides=2, activation='relu', padding='same' , name="Encoder_Conv2D_2"),
             layers.Conv2D(64, 3, strides=2, activation='relu', padding='same' , name="Encoder_Conv2D_4"),
              layers.Conv2D(128, 3, strides=2, activation='relu', padding='same' , name="Encoder_Conv2D_6"),
              layers.Flatten(name="Encoder_Flatten")
        ])

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")


    #build the Encoder
    def encoder(self,data):
        x = self.encoder(data)
        z_mean, z_log_var = self.z_mean(x), self.z_log_var(x)
        return z_mean, z_log_var

    # build the reparmeterization/sampling layer
    def reparametrization(self, z_mean, z_log_var):
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        z = z_mean + tf.exp(0.5 * z_log_var) * epsilon
        return z

    def decode(self, data):
        return self.decoder(data)

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def recon_loss(self,data,reconstruction):
        return tf.reduce_mean(binary_crossentropy(data,reconstruction))

    def kl_divergence(self, Z_logvar, Z_mu): # Corrected indentation here
        kl_loss = -0.5 * tf.reduce_mean(1 + Z_logvar - Z_mu**2 - tf.math.exp(Z_logvar))
        return self.beta*kl

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encode(data)
            z = self.reparameterization(z_mean, z_log_var)
            reconstruction = self.decode(z)